In [1]:
import numpy as np

In [2]:
def Dc_calculator(ss, kappa):
    """
    calculate critical diameter from kappa (hygroscopicity) and supersaturation
    rewrite from GK's MATLAB code

    Parameters
    ----------
    ss : float
        super saturation [%]
    kappa : float
        hygroscopicity

    Returns
    -------
    dc : float
        critical diameter [nm]

    """
    
    if np.isnan(ss) or np.isnan(kappa):
        return(np.nan)
    
    if ss<=0 or kappa<=0:
        print('WARNING: either ss or kappa input is not positive, return NaN')
        return(np.nan)
    
    deltaaa=1e-12
    A = 2.1e-09       # constant
    Ds_nm = 0.0
    
    Ds=7e-09    #dry particle dia, first initial value in meters 
    
    Critical_SS3 = -1.0
    ddd = -1.0
    
    
    Critical_SSc_store = np.zeros(60)
    Ds_store = np.zeros(60)
    
    for ii in range(0,30):
        Ds1=Ds
                 
        Dwet2 = np.arange(Ds, 1e-6, 1e-9) # initiate with D = Ds, because D or wet dia is always > than Ds
        RHS_satratiom1 = (((Dwet2**3-Ds**3)/(Dwet2**3-Ds**3*(1-kappa)))*np.exp(A/Dwet2))-1
        
        Critical_SS1= np.mean(np.max(RHS_satratiom1))*100
        Critical_SSc_store[2*ii] = Critical_SS1
               
        Ds  = Ds +  deltaaa  # increment the D value by deltaaa to find local slope    
        RHS_satratiom1 = ( ((Dwet2**3-Ds**3)/(Dwet2**3-Ds**3*(1-kappa))) * np.exp(A/Dwet2) )-1
        
        Critical_SS2= np.mean(np.max(RHS_satratiom1))*100
        
        Critical_SSc_store[2*ii+1] = Critical_SS2 # match at what i value CCNc SScrit matches, at that i value find what kappa was
        Ds_store[2*ii]=Ds1
        Ds_store[2*ii+1]=Ds
    
        dist=(ss-Critical_SS1)*(deltaaa)/(Critical_SS2-Critical_SS1)  # dist is the difference between where Ds was at the begining of the loop and where the predicted zero is.
        Ds=Ds1+dist
        
        if ii>1 and (np.abs(Critical_SS3-Critical_SS2)<1e-5 or ((dist*ddd<0) and dist<1e-12)):
            break
    
        Critical_SS3=Critical_SS2
        #This uses newton-iteration to find zeros. it tries to diminish
        #(ss_crit-Critical_SSc_store(2*i+1) quickly by approximating where the zero
        #would be based on the slope.
        ddd=dist
        
    # print(ii)
    diff = np.abs(Critical_SSc_store-ss)
    numb = np.argmin(diff)
    Ds_nm = Ds_store[numb]*1e9
    # numb = np.where(diff == diff.min())
    # if len(numb)!=1:
        # print(diff)
        # Ds_nm = np.mean(Ds_store[numb])*1e9
    # else:
        # Ds_nm = Ds_store[numb[0]]*1e9
        
    return(Ds_nm)

In [6]:
Dc_calculator(0.37, .2)

np.float64(79.30806282116042)